<a href="https://colab.research.google.com/github/23023339-JaydenPang/FYP_Rally/blob/main/Gemini_API_(Use_This).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
# === Install dependencies ===
!pip install -q google-generativeai pandas gspread gspread-dataframe beautifulsoup4

# === Imports and authentication ===
import time
import pandas as pd
from datetime import datetime
import google.generativeai as genai
from google.colab import auth, userdata
import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from google.auth import default
from googleapiclient.discovery import build
import requests
import json
from bs4 import BeautifulSoup
import re

auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# === Configure Gemini ===
gemini_key = userdata.get('GeminiAPIKey')
if not gemini_key:
    raise ValueError("❌ Gemini API key not found. Use userdata.set('GeminiAPIKey', 'your-key')")

genai.configure(api_key=gemini_key)
model = genai.GenerativeModel("gemini-2.0-flash", generation_config={
    "temperature": 0.2, "top_p": 0.9, "top_k": 50, "max_output_tokens": 800
})

# === Sheet utilities ===
def enable_text_wrap(spreadsheet_id, worksheet_name, num_columns):
    service = build('sheets', 'v4', credentials=creds)
    sheet_metadata = service.spreadsheets().get(spreadsheetId=spreadsheet_id).execute()
    sheet_id = next((s['properties']['sheetId'] for s in sheet_metadata['sheets'] if s['properties']['title'] == worksheet_name), None)
    if sheet_id is None:
        raise ValueError(f"Worksheet '{worksheet_name}' not found.")
    requests = [{
        "repeatCell": {
            "range": {
                "sheetId": sheet_id,
                "startRowIndex": 0,
                "startColumnIndex": 0,
                "endColumnIndex": num_columns
            },
            "cell": {
                "userEnteredFormat": {
                    "wrapStrategy": "WRAP"
                }
            },
            "fields": "userEnteredFormat.wrapStrategy"
        }
    }]
    service.spreadsheets().batchUpdate(
        spreadsheetId=spreadsheet_id, body={"requests": requests}
    ).execute()

def append_to_sheet(sheet_name, worksheet_name, new_df):
    try:
        spreadsheet = gc.open(sheet_name)
    except gspread.exceptions.SpreadsheetNotFound:
        spreadsheet = gc.create(sheet_name)
        spreadsheet.share('', perm_type='anyone', role='writer')
    try:
        worksheet = spreadsheet.worksheet(worksheet_name)
    except gspread.exceptions.WorksheetNotFound:
        worksheet = spreadsheet.add_worksheet(title=worksheet_name, rows="1000", cols="30")
    existing_df = get_as_dataframe(worksheet).dropna(how='all')
    updated_df = pd.concat([existing_df, new_df], ignore_index=True) if not existing_df.empty else new_df
    set_with_dataframe(worksheet, updated_df)
    enable_text_wrap(spreadsheet.id, worksheet_name, len(updated_df.columns))
    print(f"✅ Appended to: https://docs.google.com/spreadsheets/d/{spreadsheet.id}")

# === Social media extraction utility ===
def extract_social_media_links(html, base_url):
    soup = BeautifulSoup(html, "html.parser")
    links = soup.find_all("a", href=True)

    social_media = {"instagram": "Not found", "facebook": "Not found", "other": []}

    for link in links:
        href = link['href'].strip()
        if "instagram.com" in href and social_media["instagram"] == "Not found":
            social_media["instagram"] = href
        elif "facebook.com" in href and social_media["facebook"] == "Not found":
            social_media["facebook"] = href
        elif any(x in href for x in ["tiktok.com", "youtube.com", "linkedin.com", "twitter.com", "x.com"]):
            social_media["other"].append(href)

    if not social_media["other"]:
        social_media["other"] = "Not found"

    return social_media

# === LLM Prompt Tuning ===
# === Together AI Inference ===
def call_together(prompt, model_name="mistralai/Mixtral-8x7B-Instruct-v0.1", temperature=0.2, max_tokens=800):
    together_key = userdata.get("TogetherAPIKey")
    if not together_key:
        raise ValueError("❌ Together API key not found. Use userdata.set('TogetherAPIKey', 'your-key')")

    url = "https://api.together.xyz/v1/chat/completions"
    headers = {"Authorization": f"Bearer {together_key}", "Content-Type": "application/json"}
    payload = {
        "model": model_name,
        "temperature": temperature,
        "max_tokens": max_tokens,
        "messages": [{"role": "user", "content": prompt}]
    }

    start_time = time.time()
    start_timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    try:
        response = requests.post(url, headers=headers, json=payload)
        data = response.json()
        text = data["choices"][0]["message"]["content"].strip()
        prompt_tokens = data["usage"]["prompt_tokens"]
        output_tokens = data["usage"]["completion_tokens"]
        return {
            "model": model_name,
            "prompt": prompt,
            "response": text,
            "prompt_tokens": prompt_tokens,
            "output_tokens": output_tokens,
            "total_tokens": prompt_tokens + output_tokens,
            "start_time": start_timestamp,
            "end_time": datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            "response_time": round(time.time() - start_time, 2)
        }
    except Exception as e:
        return {
            "model": model_name,
            "prompt": prompt,
            "response": f"❌ Together AI error: {e}",
            "prompt_tokens": 0,
            "output_tokens": 0,
            "total_tokens": 0,
            "start_time": start_timestamp,
            "end_time": datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            "response_time": round(time.time() - start_time, 2)
        }

# === LLM Prompt Tuning with Gemini, Groq, and Together ===
def tune_prompt(prompt_for_logging, label, original_output, qna_or_review, prompt_for_generation=None):
    if prompt_for_generation is None:
        prompt_for_generation = prompt_for_logging

    results = []
    start_time = time.time()
    start_timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

    # --- Original Gemini Output (uses prompt_for_generation)
    input_str = f"{prompt_for_generation}\n\nAnswer: \"{qna_or_review}\""
    try:
        gemini_out = model.generate_content(input_str).text.strip()
        gemini_tokens = model.count_tokens(gemini_out).total_tokens
        prompt_tokens = model.count_tokens(prompt_for_logging).total_tokens
    except Exception as e:
        gemini_out, gemini_tokens, prompt_tokens = f"❌ Gemini error: {e}", 0, 0

    results.append({
        "Model": f"Gemini (Original {label} Prompt)",
        "Input Prompt": prompt_for_logging,
        "Input": qna_or_review,
        "Response": gemini_out,
        "Input Tokens": prompt_tokens,
        "Output Tokens": gemini_tokens,
        "Total Tokens": prompt_tokens + gemini_tokens,
        "Start Time": start_timestamp,
        "End Time": datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        "Response Time (s)": round(time.time() - start_time, 2)
    })

    # --- Together AI Prompt Improvement → Gemini
    if use_together:
        try:
            optimization_instruction = (
                "Improve the following prompt to generate more accurate and complete responses from Gemini, "
                "while reducing the total token cost. Keep the original output structure and required JSON keys unchanged. "
                "Remove any unnecessary words or repetition. Avoid verbose instructions. Be concise but clear.\n\n"
            )
            together_result = call_together(optimization_instruction + prompt_for_logging)
            improved_prompt = together_result["response"]
            improved_input = f"{improved_prompt}\n\nAnswer: \"{qna_or_review}\""
            gemini_out_together = model.generate_content(improved_input).text.strip()
            gemini_tokens = model.count_tokens(gemini_out_together).total_tokens
            prompt_tokens = model.count_tokens(improved_prompt).total_tokens
            results.append({
                "Model": f"Gemini (Together-Tuned {label} Prompt)",
                "Input Prompt": improved_prompt,
                "Input": qna_or_review,
                "Response": gemini_out_together,
                "Input Tokens": prompt_tokens,
                "Output Tokens": gemini_tokens,
                "Total Tokens": prompt_tokens + gemini_tokens,
                "Start Time": together_result["start_time"],
                "End Time": datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                "Response Time (s)": together_result["response_time"]
            })
        except Exception as e:
            print(f"❌ Together AI error for {label} Prompt: {e}")

    # --- Groq Prompt Improvement → Gemini
    if use_groq:
        try:
            groq_key = userdata.get("GroqAPIKey")
            optimization_instruction = (
                "Improve the following prompt to generate more accurate and complete responses from Gemini, "
                "while reducing the total token cost. Keep the original output structure and required JSON keys unchanged. "
                "Remove any unnecessary words or repetition. Avoid verbose instructions. Be concise but clear.\n\n"
            )
            groq_resp = requests.post(
                "https://api.groq.com/openai/v1/chat/completions",
                headers={"Authorization": f"Bearer {groq_key}", "Content-Type": "application/json"},
                json={
                    "model": "llama3-8b-8192",
                    "messages": [{"role": "user", "content": optimization_instruction + prompt_for_logging}],
                    "temperature": temperature, "max_tokens": 800
                }
            ).json()
            improved_prompt = groq_resp["choices"][0]["message"]["content"]

            if label == "Business":
                res = requests.get(qna_or_review, headers={"User-Agent": "Mozilla/5.0"})
                page_html = res.text
                social_links = extract_social_media_links(page_html, qna_or_review)
                enriched_prompt = (
                    f"{improved_prompt}\n\n"
                    f"## Extracted Social Media Links:\n{json.dumps(social_links, indent=2)}\n\n"
                    f"## Truncated HTML Content:\n{page_html[:10000]}"
                )
            else:
                enriched_prompt = f"{improved_prompt}\n\nAnswer: \"{qna_or_review}\""

            gemini_out_groq = model.generate_content(enriched_prompt).text.strip()
            gemini_tokens_groq = model.count_tokens(gemini_out_groq).total_tokens
            prompt_tokens = model.count_tokens(improved_prompt).total_tokens
            results.append({
                "Model": f"Gemini (Groq-Tuned {label} Prompt)",
                "Input Prompt": improved_prompt,
                "Input": qna_or_review,
                "Response": gemini_out_groq,
                "Input Tokens": prompt_tokens,
                "Output Tokens": gemini_tokens_groq,
                "Total Tokens": prompt_tokens + gemini_tokens_groq,
                "Start Time": start_timestamp,
                "End Time": datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                "Response Time (s)": round(time.time() - start_time, 2)
            })
        except Exception as e:
            print(f"❌ Groq error for {label} Prompt: {e}")

    return pd.DataFrame(results)



# === User Inputs ===
review_text = "question\": \"What improvement would you like to see in our OG Gold membership?\", \t\t\t\"question_type\": \"long\",       \"answer\": \"I don’t have anything in particular to suggest. Everything has been very very very very good so far with the OG Gold membership. I think is good and don't need to change at the moment.\""  #@param {type:"string"}
golden_prompt = "As Rally AI's evaluation assistant, evaluate answers with:  - Usefulness_score: 0-2 (poor, med, good) - Usefulness_reason (<21 words) - Authenticity_score: 0-5 (not gen, maybe gen, gen) - Authenticity_reason (<21 words)  Identify answers to improve with low usefulness or authenticity scores.  Output JSON:  ```json {   \"review\": {     \"Question 1\": {       \"Usefulness_score\": 0,       \"Usefulness_reason\": \"\",       \"Authenticity_score\": 0,       \"Authenticity_reason\": \"\"     },     \"...\": {       ...     }   },   \"Answers_to_improve\": [\"Question 2\", ...],   \"Feedback\": \"\" } ```"  #@param {type:"string"}
consumer_profiling_prompt = ""  #@param {type:"string"}
business_profiling_prompt = ""  #@param {type:"string"}
consumer_qna = ""  #@param {type:"string"}
company_link = ""  #@param {type:"string"}
use_groq = True  #@param {type:"boolean"}
use_together = True #@param {type:"boolean"}
show_all_outputs = True  #@param {type:"boolean"}
log_to_sheets = True  #@param {type:"boolean"}
temperature = 0.2  #@param {type:"slider", min:0.0, max:1.0, step:0.1}

# === Business prompt enhancement with scraped web data ===
try:
    headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36"
}
    res = requests.get(company_link, headers=headers, timeout=10)

    if res.status_code == 200:
        page_html = res.text
        social_links = extract_social_media_links(page_html, company_link)
        social_json = json.dumps(social_links, indent=2)
        business_prompt_with_links = (
            f"{business_profiling_prompt}\n\n"
            f"## Extracted Social Media Links:\n{social_json}\n\n"
            f"## Truncated HTML Content:\n{page_html[:10000]}"
        )
    else:
        print(f"❌ Failed to load website: {company_link}")
        business_prompt_with_links = business_profiling_prompt
except Exception as e:
    print(f"❌ Error fetching company page: {e}")
    business_prompt_with_links = business_profiling_prompt

# Keep original prompt separate for correct logging
business_prompt_only = business_profiling_prompt

# === Run prompt evaluations only if input is provided ===
eval_df = pd.DataFrame()
consumer_df = pd.DataFrame()
business_df = pd.DataFrame()

if review_text.strip():
    eval_df = tune_prompt(golden_prompt, "Golden", "", review_text)

if consumer_qna.strip():
    consumer_df = tune_prompt(consumer_profiling_prompt, "Consumer", "", consumer_qna)

if company_link.strip():
    business_df = tune_prompt(
        prompt_for_logging=business_prompt_only,
        label="Business",
        original_output="",
        qna_or_review=company_link,
        prompt_for_generation=business_prompt_with_links
    )



# === Log results only if corresponding input is not empty ===
if log_to_sheets:
    if not eval_df.empty:
        append_to_sheet("Gemini_Responses", "Sheet1", eval_df)
    if not consumer_df.empty:
        append_to_sheet("Consumer Profiling", "Sheet1", consumer_df)
    if not business_df.empty:
        append_to_sheet("Business Profiling", "Sheet1", business_df)


# === Display ===
if show_all_outputs:
    print("\n📊 Prompt Evaluation Summary")
    for df in [eval_df, consumer_df, business_df]:
        for row in df.itertuples():
            print(f"\n==== {row.Model} ====")
            print(f"Prompt: {row._2[:200]}...\nResponse: {row.Response[:500]}")

❌ Error fetching company page: Invalid URL '': No scheme supplied. Perhaps you meant https://?
✅ Appended to: https://docs.google.com/spreadsheets/d/1v10PU7lJ0BLs45zE98DtAE9wkgwKXZi925u2YGD_CV4

📊 Prompt Evaluation Summary

==== Gemini (Original Golden Prompt) ====
Prompt: As Rally AI's evaluation assistant, evaluate answers with:  - Usefulness_score: 0-2 (poor, med, good) - Usefulness_reason (<21 words) - Authenticity_score: 0-5 (not gen, maybe gen, gen) - Authenticity...
Response: ```json
{
  "review": {
    "What improvement would you like to see in our OG Gold membership?": {
      "Usefulness_score": 0,
      "Usefulness_reason": "The answer doesn't provide any suggestions for improvement, rendering it unhelpful.",
      "Authenticity_score": 5,
      "Authenticity_reason": "The response sounds like a genuine user experience, expressing satisfaction."
    }
  },
  "Answers_to_improve": [
    "What improvement would you like to see in our OG Gold membership?"
  ],
  "Fe

==== Gemin

In [ ]:
# === Install dependencies ===
!pip install -q google-generativeai pandas gspread gspread-dataframe

# === Imports and authentication ===
import time
import pandas as pd
from datetime import datetime
import google.generativeai as genai
from google.colab import auth, userdata
import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from google.auth import default
from googleapiclient.discovery import build

# === Authenticate ===
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# === Configure Gemini ===
gemini_key = userdata.get('GeminiAPIKey')
if not gemini_key:
    raise ValueError("❌ Gemini API key not found. Use userdata.set('GeminiAPIKey', 'your-key')")

genai.configure(api_key=gemini_key)
model = genai.GenerativeModel("gemini-2.0-flash", generation_config={
    "temperature": 0.2, "top_p": 0.9, "top_k": 50, "max_output_tokens": 800
})

def enable_text_wrap(spreadsheet_id, sheet_name, num_columns):
    service = build('sheets', 'v4', credentials=creds)
    sheet_metadata = service.spreadsheets().get(spreadsheetId=spreadsheet_id).execute()
    sheet_id = next(s['properties']['sheetId'] for s in sheet_metadata['sheets'] if s['properties']['title'] == sheet_name)

    requests = [{
        "repeatCell": {
            "range": {
                "sheetId": sheet_id,
                "startRowIndex": 0,
                "endColumnIndex": num_columns
            },
            "cell": {
                "userEnteredFormat": {
                    "wrapStrategy": "WRAP"
                }
            },
            "fields": "userEnteredFormat.wrapStrategy"
        }
    }]
    service.spreadsheets().batchUpdate(spreadsheetId=spreadsheet_id, body={"requests": requests}).execute()


# === Sheet utilities ===
def append_to_sheet(sheet_name, worksheet_name, new_df):
    try:
        spreadsheet = gc.open(sheet_name)
        created = False
    except gspread.SpreadsheetNotFound:
        spreadsheet = gc.create(sheet_name)
        spreadsheet.share('', perm_type='anyone', role='writer')
        created = True

    try:
        worksheet = spreadsheet.worksheet(worksheet_name)
    except gspread.WorksheetNotFound:
        worksheet = spreadsheet.add_worksheet(title=worksheet_name, rows=100, cols=20)

    existing_df = get_as_dataframe(worksheet, evaluate_formulas=True).dropna(how='all')
    updated_df = pd.concat([existing_df, new_df], ignore_index=True)
    set_with_dataframe(worksheet, updated_df)

    # ✅ Wrap text in all columns
    enable_text_wrap(spreadsheet.id, worksheet_name, len(updated_df.columns))

    sheet_url = f"https://docs.google.com/spreadsheets/d/{spreadsheet.id}"
    if created:
        print(f"📄 Created new Google Sheet: {sheet_url}")
    else:
        print(f"📄 Updated existing Google Sheet: {sheet_url}")
    print(f"🔗 View Full Results: {sheet_url}")
    return spreadsheet.id


# === User Inputs ===
qna_profile = " Jayden’ Responses What is your age group? → 18-24  What is your gender? → Male  Where do you live? → Yew Tee, Singapore  What is your occupation? → Student  What is your highest level of education? → Polytechnic Diploma  What is your monthly household income bracket? → S$4,000–S$6,000   Dining Behaviour How often do you dine out each week? → 3–4 times a week  How much do you usually spend on a typical meal? → Around S$12–S$20  How do you usually choose where to eat? → I read Google or TikTok reviews and check what’s trending.  Do you prefer dining alone, with friends, or family? → Mostly with friends  How far are you willing to travel for a good meal? → Up to 30 minutes on MRT   Food Preferences & Psychographics What are your top 3 favourite cuisines? → Japanese, Thai, and Korean  Are there any cuisines you avoid or dislike? → Not a big fan of Mediterranean food  How sensitive are you to food prices when choosing a place to eat? → Somewhat sensitive — I don’t like overpaying unless it’s worth it  What matters most when dining out? → Taste and value for money  How likely are you to try new foods or restaurants? → Very likely — I enjoy discovering new places   Engagement & Tech Use How often do you leave reviews or feedback after eating out? → Occasionally, when the experience is really good or bad  Do you use food delivery apps or dining loyalty programmes? → Yes, I use GrabFood and occasionally Chope for bookings  How often do you complete quests on Rally? → Once or twice a month  What motivates you to complete quests? → Mainly the rewards and to try new places with discounts"  #@param {type:"string", label:"Consumer QnA Profile"}
behaviour_questions = "Behavioral Questions When choosing a place to eat, what platforms or tools do you usually use to research or decide? (e.g., Google reviews, TikTok, Instagram, food blogs, word of mouth)  How do your dining preferences change depending on who you're eating with (friends, family, alone)? (e.g., spend more, choose different cuisine, go to casual vs formal spots)  Describe a recent time when you were impressed or disappointed by a dining experience. What made it memorable? (Look for triggers of satisfaction or dissatisfaction)  What influences you more when trying a new restaurant: social media trends or personal recommendations? Why?  Have you ever returned to a restaurant because of a loyalty reward or promotion? What was the offer?  When faced with two restaurants—one more expensive but better-rated, and one cheaper but average—which would you choose and why?  How do you usually discover new food spots? Do you actively search, or find them spontaneously?  Tell us about a time you tried a cuisine you weren’t familiar with. What motivated you to try it? (e.g., curiosity, peer influence, trending)  When using food apps like GrabFood or Chope, what features do you value the most? (e.g., ease of use, discounts, ratings, location filters)  If you received a mission/quest offering 30% off at a new restaurant 25 minutes away, what factors would influence your decision to go or not? (e.g., travel time, cuisine, price, company, timing)"  #@param {type:"string", label:"Behavioural Questions"}

# === Generate and Log Gemini Answer ===
result_df = pd.DataFrame()
sheet_url = ""

if qna_profile.strip() and behaviour_questions.strip():
    input_prompt = (
        "You are Rally AI. Based on the following consumer profile (QnA), answer the behavioural questions that follow.\n\n"
        f"## QnA Profile:\n{qna_profile}\n\n"
        f"## Behavioural Questions:\n{behaviour_questions}"
    )

    start_time = time.time()
    start_timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

    try:
        response = model.generate_content(input_prompt).text.strip()
        output_tokens = model.count_tokens(response).total_tokens
        input_tokens = model.count_tokens(input_prompt).total_tokens
    except Exception as e:
        response = f"❌ Gemini error: {e}"
        output_tokens = 0
        input_tokens = 0

    result_df = pd.DataFrame([{
        "QnA Profile": qna_profile,
        "Behavioural Questions": behaviour_questions,
        "Gemini Response": response,
        "Input Tokens": input_tokens,
        "Output Tokens": output_tokens,
        "Start Time": start_timestamp,
        "End Time": datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        "Response Time (s)": round(time.time() - start_time, 2)
    }])

    sheet_url = append_to_sheet("consumer_testing", "Sheet1", result_df)

# === Display ===
if not result_df.empty:
    print("\n📊 Gemini Response Summary:")
    print(result_df[['Gemini Response']].iloc[0][0][:1000])
    print(f"\n🔗 View Full Results: {sheet_url}")
